In [1]:
!pip install matplotlib

In [2]:
import os
import sys
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print("done")
print(device)

done
cpu


In [3]:
os.getcwd()

'/workspace'

In [4]:
HOMEDIR = os.getcwd()
if "jovyan" in HOMEDIR:
    HOMEDIR = "/home/jovyan/sivtsov/graph_agent"

In [5]:
# crutch for code visibiltiy
sys.path.append(HOMEDIR)

In [6]:
STARPU_HOME = HOMEDIR

# Inspect

here you can dump whole graph, to see how it looks like. It is traced from nntile model (which created from pytorch). So operations should be as in original model (there are tests)

In [7]:
import os
import json
from subprocess import Popen, PIPE
from simulator.nnt_graph_sched_utils import (
    BufferizedIterator, 
    StreamGraphLinesCatcher,
    GraphParser,
    SchedulerTaskGraph,
    TimingsParser,
    PrioritySerializer
)

In [8]:
from graph_trainer import (
    construct_program_cmd,
    custom_dynamic_policy, 
    custom_random_policy,
    GraphRunner
)

In [9]:
file = f"{HOMEDIR}/graph_tracer/traced_graph_2_small_layers.json"
ncpus = 1
ncuda = 0
niters = 1
manual_sampling = False

verbose = True

gr = GraphRunner(file, ncpus, ncuda, niters, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
gr.run(interactive=False, starpu_home=STARPU_HOME)

Run process:  python3 /workspace/graph_dataset.py --file /workspace/graph_tracer/traced_graph_2_small_layers.json --ncpus 1 --ncuda 0 --niters 1
Process pid: 527


In [10]:
_ = gr.skip_initialization_routine()
graph_init = gr.next()

In [11]:
# https://graphviz.org/
gi_gv = graph_init.to_graphviz()

In [12]:
gi_gv.render(f"{HOMEDIR}/small.dot")

'/workspace/small.dot.pdf'

# Slice the graph

For better convergence, debug and replay buffer purposes, you have the ability to provide slice indicies for graph

Note, you should use interactive mode 

In [13]:
file = f"{HOMEDIR}/graph_tracer/traced_graph_2_small_layers.json"
ncpus = 1
ncuda = 0
niters = 1
manual_sampling = True

verbose = True

gr = GraphRunner(file, ncpus, ncuda, niters, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
gr.run(interactive=False, starpu_home=STARPU_HOME)

Run process:  python3 /workspace/graph_dataset.py --file /workspace/graph_tracer/traced_graph_2_small_layers.json --ncpus 1 --ncuda 0 --niters 1 --manual_sampling
Process pid: 536


In [14]:
with open(file, "r") as f:
    config = json.load(f)

operations = config["calls"]
total_operations = len(operations)
total_operations

141

In [15]:
_ = gr.skip_initialization_routine()

In [16]:
graph_slice = gr.next([50,60])

In [17]:
# note that where are actually also initialization ops 
len(graph_slice.nodes)

14

# Collect statistics

First, you need to simulate graph multiple times to collect starpu statistics (nntile based on starpu)   
It will give you timings for every operation in graph

Note, they are statistically collected and will not be updated during next simulations runs. So you can't train on them. 

In [18]:
# file = f"{HOMEDIR}/graph_tracer/traced_graph_2_small_layers.json"
# ncpus = 4
# ncuda = 0
# niters = 25
# manual_sampling = False

# verbose = True

# gr = GraphRunner(file, ncpus, ncuda, niters, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
# gr.run(interactive=False, collect_metrics=True)
# gr.p.wait()

In [19]:
# file = f"{HOMEDIR}/graph_tracer/traced_graph_8_layers.json"
# ncpus = 4
# ncuda = 1
# niters = 25
# manual_sampling = False

# verbose = True

# gr = GraphRunner(file, ncpus, ncuda, niters, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
# gr.run(interactive=False, collect_metrics=True, starpu_home=STARPU_HOME)
# gr.p.wait()

In [20]:
# file = f"{HOMEDIR}/graph_tracer/traced_graph_4_layers.json"
# ncpus = 4
# ncuda = 0
# niters = 25
# manual_sampling = False

# verbose = True

# gr = GraphRunner(file, ncpus, ncuda, niters, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
# gr.run(interactive=False, collect_metrics=True, starpu_home=STARPU_HOME)
# gr.p.wait()

In [21]:
# file = f"{HOMEDIR}/graph_tracer/traced_graph_16_layers.json"
# ncpus = 4
# ncuda = 0
# niters = 25
# manual_sampling = False

# verbose = True

# gr = GraphRunner(file, ncpus, ncuda, niters, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
# gr.run(interactive=False, collect_metrics=True, starpu_home=STARPU_HOME)
# gr.p.wait()

In [22]:
# # But if all iterations completed - should be fine
# gr.stderr_iter.buffer[-50:]

# Agent

here I suggest you to decouple some logic   

In [23]:
max_gr_width:int = 512
num_device:int  = ncpus + ncuda
device_metr:int = 1
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 512)
        self.layer2 = nn.Linear(512, 512)
        self.layer3 = nn.Linear(512, 512)
        self.layer4 = nn.Linear(512, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        return self.layer4(x)



In [24]:
def initialize():
    name_op_to_id = {}
    gr = GraphRunner(f"{HOMEDIR}/graph_tracer/traced_graph_16_layers.json", ncpus, ncuda, 1, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
    gr.run()
    print(gr)
    _ = gr.skip_initialization_routine()
    graph = gr.next()
    for i, (_, n) in enumerate(graph.nodes.items()):
      if n.name != 'NULL':
        name_op_to_id[n.name] = -1
    id:int = 0
    for i, (_, n) in enumerate(graph.nodes.items()):
        if n.name != 'NULL':
            if (name_op_to_id[n.name] == -1):
                name_op_to_id[n.name] = id
                id += 1
    gr.serialize_priorities(graph)
    timings = gr.wait_get_timings()
    return [id, name_op_to_id]
[col_ops, name_op_to_id] = initialize()
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.001
EPS_END = 0.00001
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4
NODE_METRIKS = 4
len_operations_data:int = col_ops + NODE_METRIKS
SIZE_BACKET = 30
MEMORY_SIZE = 10000
EPOCH_BEFORE_BEST_REC = 20

Run process:  python3 /workspace/graph_dataset.py --file /workspace/graph_tracer/traced_graph_16_layers.json --ncpus 1 --ncuda 0 --niters 1 --manual_sampling
Process pid: 542
n_nodes: 1055 serialized: 1
Wait timings
Compute timings: t_begin=4319484436044, t_end=4325244326644, delta_ms=5759.8906


In [25]:
class graph_slice:
  free_nodes = []
  cnt_in = {}
  deeps = {}
  gr = []
  fr_t = []
  allgr = []
  cur_prio = int(1e9)
  load_of_dev = []
  def __init__(self, gr) -> None:
      global num_device
      graph = gr.next()
      self.free_nodes = []
      self.cnt_in = {}
      self.load_of_dev = [0] * num_device
      self.sum_time = 0
      self.sum_dim = 0
      
      print(len(graph.nodes.items()))
      for i, (_, n) in enumerate(graph.nodes.items()):
        if n.name != 'NULL':
            self.cnt_in[n.addr] = len(n.inc)
            if (len(n.inc) == 0):
              self.free_nodes.append(n.addr)
      print("graph_gened")
      self.cur_prio = int(1e9)
      self.gr = graph.nodes
      for el in self.free_nodes:
          self.deeps[el] = 0
      self.fr_t = gr
      self.allgr = graph
      self.last_measure = 0
      self.dlt = 0
      for i, (_, n) in enumerate(graph.nodes.items()):
        if n.name != 'NULL':
            self.sum_time += self.gr[n.addr].perfmodels[0].length_mcs
  def make_decision(self, v:int, mask_dev) -> bool:
      if (v >= len(self.free_nodes)):
          return False
      v = self.free_nodes[v]
      self.gr[v].prio = self.cur_prio
      self.gr[v].device_affinity = mask_dev
      self.cur_prio-=1
      
      self.free_nodes.remove(v)
      prob_used = []
      for  i in range(num_device):
          prob_used.append(i)
      for el in prob_used:
          self.load_of_dev[el] += (self.gr[v].perfmodels[0].length_mcs / self.sum_time / len(prob_used))
      for el in  self.gr[v].out:
        if (self.gr[el].name != 'NULL'):
            self.cnt_in[el]-=1
            if (self.cnt_in[el] == 0):
              self.deeps[el] = 0
              for u in self.gr[el].inc:
                  self.deeps[el] = max(self.deeps[el], self.deeps[v] + 1)
              self.free_nodes.append(el)
      return True

  def get_el_data(self, v:int, l:int, res):
    global name_op_to_id, col_ops
    res[l + name_op_to_id[self.gr[v].name]] = 1
    res[l + col_ops] = self.deeps[v] / len(self.gr)
    res[l + col_ops + 1] = len(self.gr[v].out) / len(self.gr)
    res[l + col_ops + 2] = self.gr[v].perfmodels[0].length_mcs/ self.sum_time
  def get_dev_data(self, j:int, l:int, res):
    res[l] = self.load_of_dev[j] 
  def get_state_vector(self):
    global max_gr_width, len_operations_data, col_ops
    res = np.zeros(max_gr_width * len_operations_data + num_device * device_metr)
    l = 0;
    for el in self.free_nodes:
      self.get_el_data(el, l, res)
      l += len_operations_data
    for j in range(num_device):
      self.get_dev_data(j, l, res)
      l += device_metr
    return res
  def get_timings(self):
    while (len(self.free_nodes)> 0):
        self.make_decision(0, 0)
    self.allgr.nodes = self.gr
    self.fr_t.serialize_priorities(self.allgr)
    cur_t = self.fr_t.wait_get_timings()
    print(self.last_measure)
    print(cur_t)
    self.dlt = self.last_measure - cur_t
    self.last_measure = cur_t
    return cur_t
  def get_time(self, v:int):
    return (-self.last_measure * self.gr[v].perfmodels[0].length_mcs/ self.sum_time)


In [26]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [27]:
def gen_pair(sz:int):
    res = np.random.randint(0, sz, 2)
    if (res[0] > res[1]):
        swap(res[0], res[1])

In [28]:
class BaselineAgent:
    def __init__(self, strategy="random", len_operations_data = 0, max_gr_width = 0, num_device = 0, device="cpu"):
        """
        here you can store actual model
        and history of this agent (maybe resent, not whole)
        """
        # self.agent_decisions = []
        self.strategy = strategy
        self.device = device
        self.policy_net = DQN(len_operations_data * max_gr_width + num_device * device_metr, max_gr_width * num_device).to(device)
        self.target_net = DQN(len_operations_data * max_gr_width + num_device * device_metr, max_gr_width * num_device).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.memory = ReplayMemory(MEMORY_SIZE)
        self.steps_done = 0
        self.num_device = num_device
    def select_action(self, state, width:int = max_gr_width):
        sample = random.random()
        global EPS_END, EPS_START, EPS_DECAY
        eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * self.steps_done / EPS_DECAY)
        self.steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
                res = self.policy_net(state).max(1).indices.view(1, 1)
                return res
        else:
        #print("rd")
            return torch.tensor([[random.randint(0, width * num_device - 1)]])
    def get_values(self, action):
        res:int = action[0][0]
        msk = [False] * self.num_device
        msk[res % self.num_device] = 1
        return [res // self.num_device, msk]

    def schedule(self, graph):
        """
        Here you just schedule train
        Don't do expensive grapdient/state computations, just simulation step
        """
        # custom_dynamic_policy/custom_random_policy. Look at graph_trainer.py
        # First baseline you should beat is custom_random_policy. Because even simplest DFS is better
        # Second - custom_dynamic_policy - DFS like approach. 
        # Probably you will just learn this strategy with your agent for now. 
        # To beat it, graph should be more complex, not strategy
        if self.strategy=="dynamic":
            custom_dynamic_policy(i, graph, strategy=self.device)
            return
        elif self.strategy=="random":
            custom_random_policy(i, graph, strategy=self.device)
            return
        elif self.strategy == "neural":
            pass
        else:
            return
        my_slice = graph
        state = torch.tensor(my_slice.get_state_vector(), dtype=torch.float32, device=self.device).unsqueeze(0)
        self.frc = []
        good_steps:int = 0
        steps = 0
        pred = []
        for t in count():
            # print("trouble", my_slice.free_nodes, pred)
            assert my_slice.free_nodes != pred, "fffffff"
            pred = my_slice.free_nodes.copy()
            if (len(my_slice.free_nodes) == 0):
                break;
            steps += 1
            action = self.select_action(state=state, width = len(my_slice.free_nodes))
            [v, msk] = self.get_values(action)
            #msk = [1,1,1,1]
            #print(v, msk)
            tm = -1
            if (len(my_slice.free_nodes) > v):
                tm = my_slice.free_nodes[v]
            good:bool = my_slice.make_decision(v, msk)
            #print(good)
            nxt_state = torch.tensor(np.zeros(len_operations_data * max_gr_width + num_device * device_metr), dtype=torch.float32, device=self.device).unsqueeze(0)
            if (good):
                good_steps += 1
                nxt_state = torch.tensor(my_slice.get_state_vector(), dtype=torch.float32, device=device).unsqueeze(0)
                self.frc.append([state, action, nxt_state, tm])
            else:
                self.memory.push(state, action, nxt_state, torch.tensor(-1, dtype=torch.float32, device=device).unsqueeze(0))
                action = torch.tensor([[random.randint(0, len(my_slice.free_nodes) * num_device - 1)]])
                [v, msk] = self.get_values(action)
                msk = [1,1,1,1]
                tmp = my_slice.make_decision(v, msk)
                assert tmp, "random op should be good"
                nxt_state = torch.tensor(my_slice.get_state_vector(), dtype=torch.float32, device=device).unsqueeze(0)
            state = nxt_state
        print("good=", good_steps / steps)
        return good_steps / steps
#
 #       # Perform one step of the optimization (on the policy network)
#
 #       # Soft update of the target network's weights
  #      # θ′ ← τ θ + (1 −τ )θ′

class AgentTrainer:
    def __init__(self,agent,  BATCH_SIZE = 128):
        # self.optimizer = optimizer
        # self.replay_buffer = replay_buffer
        self.BATCH_SIZE = BATCH_SIZE
        self.optimizer = optim.Adam(agent.policy_net.parameters(), lr=LR, amsgrad=True)
    
    def step(self,agent, timing, device="cpu"):
        """
        Called after getting timings from your agent decision
        Here you can maintain optimizer states, compute grads, backpropagate and so on
        Probably - here can be stored state for RL algos like replay buffer
        """
        # Called after 
        if (len(agent.memory) < self.BATCH_SIZE):
            return
        transitions = agent.memory.sample(BATCH_SIZE)
        batch = Transition(*zip(*transitions))

        # Compute a mask of non-final states and concatenate the batch elements
        # (a final state would've been the one after which simulation ended)
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)

        # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
        # columns of actions taken. These are the actions which would've been taken
        # for each batch state according to policy_net
        state_action_values = agent.policy_net(state_batch).gather(1, action_batch)

        # Compute V(s_{t+1}) for all next states.
        # Expected values of actions for non_final_next_states are computed based
        # on the "older" target_net; selecting their best reward with max(1).values
        # This is merged based on the mask, such that we'll have either the expected
        # state value or 0 in case the state was final.
        next_state_values = torch.zeros(self.BATCH_SIZE, device=device)
        with torch.no_grad():
            next_state_values[non_final_mask] = agent.target_net(non_final_next_states).max(1).values
        # Compute the expected Q values
        expected_state_action_values = (next_state_values * GAMMA) + reward_batch

        # Compute Huber loss
        criterion = nn.SmoothL1Loss()
        loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        # In-place gradient clipping
        torch.nn.utils.clip_grad_value_(agent.policy_net.parameters(), 100)
        self.optimizer.step()
        global TAU
        agent.target_net_state_dict = agent.target_net.state_dict()
        agent.policy_net_state_dict = agent.policy_net.state_dict()
        for key in agent.policy_net_state_dict:
            agent.target_net_state_dict[key] = agent.policy_net_state_dict[key]*TAU + agent.target_net_state_dict[key]*(1-TAU)
        agent.target_net.load_state_dict(agent.target_net_state_dict)

# Runner


In [29]:
from graph_trainer import (
    construct_program_cmd,
    custom_dynamic_policy, 
    custom_random_policy,
    GraphRunner
)

from interpretator import (
    get_subset_inds
)

import json
import numpy as np

2025-06-17 15:30:50.684957: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
# from graph_trainer import (
#     construct_program_cmd,
#     custom_dynamic_policy, 
#     custom_random_policy,
#     GraphRunner
# )

# from interpretator import (
#     get_subset_inds
# )

# import json
# import numpy as np
# import os
# import sys
# import math
# import random
# import matplotlib
# import matplotlib.pyplot as plt
# from collections import namedtuple, deque
# from itertools import count

# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F


# device = torch.device(
#     "cuda" if torch.cuda.is_available() else
#     "mps" if torch.backends.mps.is_available() else
#     "cpu"
# )
# print("done")
# print(device)
# HOMEDIR = os.getcwd()
# if "jovyan" in HOMEDIR:
#     HOMEDIR = "/home/jovyan/sivtsov/graph_agent"
# sys.path.append(HOMEDIR)
# STARPU_HOME = HOMEDIR
# ncpus = 4
# ncuda = 0
# manual_sampling = False

# verbose = True
# file = f"{HOMEDIR}/graph_tracer/traced_graph_2_small_layers.json"
# gr = GraphRunner(file, ncpus, ncuda, 100, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
# gr.run(interactive=True, starpu_home=STARPU_HOME)
# _ = gr.skip_initialization_routine()
# for i in range(gr.niters):
#     # inds = get_subset_inds(rng, total_operations)
#     inds = "all" # just use this if you want whole graph
#     print(f"{i=} {inds=}")
#     graph = gr.next(inds=inds)
#     custom_dynamic_policy(i, graph, strategy="cpu")
#     # print("Nodes: ", len(graph.nodes))

#     gr.serialize_priorities(graph)
#     timings = gr.wait_get_timings()


In [31]:
def get_etalon(file, gr):
    agent = BaselineAgent(strategy="dynamic", device="cpu" if ncuda==0 else "cuda")
    trainer = AgentTrainer(agent)
    timings_dynamic = []

    for i in range(5):
        # inds = get_subset_inds(rng, total_operations)
        inds = "all" # just use this if you want whole graph
        print(f"{i=} {inds=}")
        graph = gr.next()

        # print("Nodes: ", len(graph.nodes))

        agent.schedule(graph)

        gr.serialize_priorities(graph)
        timings = gr.wait_get_timings()

        trainer.step(agent, timings)
    
        timings_dynamic.append(timings)
    return timings_dynamic[-1]
    

In [32]:
file = f"{HOMEDIR}/graph_tracer/traced_graph_2_small_layers.json"
ncpus = 4
ncuda = 0
niters = 10
manual_sampling = True

verbose = True
# verbose = False

In [33]:
with open(file, "r") as f:
    config = json.load(f)

operations = config["calls"]
total_operations = len(operations)
total_operations

141

In [34]:
agent = BaselineAgent(strategy="dynamic", device="cpu" if ncuda==0 else "cuda")
trainer = AgentTrainer(agent)

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [35]:
gr = GraphRunner(file, ncpus, ncuda, niters, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
gr.run(interactive=True, starpu_home=STARPU_HOME)
_ = gr.skip_initialization_routine()

Run process:  python3 /workspace/graph_dataset.py --file /workspace/graph_tracer/traced_graph_2_small_layers.json --ncpus 4 --ncuda 0 --niters 10 --manual_sampling
Process pid: 551


In [36]:
rng = np.random.default_rng()

In [37]:
timings_dynamic = []

for i in range(gr.niters):
    # inds = get_subset_inds(rng, total_operations)
    inds = "all" # just use this if you want whole graph
    print(f"{i=} {inds=}")
    graph = gr.next(inds=inds)

    # print("Nodes: ", len(graph.nodes))

    agent.schedule(graph)

    gr.serialize_priorities(graph)
    timings = gr.wait_get_timings()

    trainer.step(agent, timings)
    
    timings_dynamic.append(timings)

i=0 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4333847946817, t_end=4333958535948, delta_ms=110.589131
i=1 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4333964803205, t_end=4334075891149, delta_ms=111.087944
i=2 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4334081066503, t_end=4334231199524, delta_ms=150.133021
i=3 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4334238517947, t_end=4334372113185, delta_ms=133.595238
i=4 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4334377746055, t_end=4334504204299, delta_ms=126.458244
i=5 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4334509986978, t_end=4334641580521, delta_ms=131.593543
i=6 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4334647664975, t_end=4334779216579, delta_ms=131.551604
i=7 inds='all'
n_nodes: 145 serial

In [38]:
agent = BaselineAgent(strategy="random", device="cpu" if ncuda==0 else "cuda")
trainer = AgentTrainer(agent)

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [39]:
gr = GraphRunner(file, ncpus, ncuda, niters, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
gr.run(interactive=True, starpu_home=STARPU_HOME)
_ = gr.skip_initialization_routine()

Run process:  python3 /workspace/graph_dataset.py --file /workspace/graph_tracer/traced_graph_2_small_layers.json --ncpus 4 --ncuda 0 --niters 10 --manual_sampling
Process pid: 559


In [40]:
timings_random = []

for i in range(gr.niters):
    # inds = get_subset_inds(rng, total_operations)
    inds = "all" # just use this if you want whole graph
    print(f"{i=} {inds=}")
    graph = gr.next(inds=inds)

    # print("Nodes: ", len(graph.nodes))

    agent.schedule(graph)

    gr.serialize_priorities(graph)
    timings = gr.wait_get_timings()

    trainer.step(agent, timings)
    
    timings_random.append(timings)

i=0 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4340827845953, t_end=4340940344848, delta_ms=112.498895
i=1 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4340946042048, t_end=4341056231952, delta_ms=110.189904
i=2 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4341061401315, t_end=4341174063845, delta_ms=112.66253
i=3 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4341179503594, t_end=4341288661046, delta_ms=109.157452
i=4 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4341293960422, t_end=4341404591512, delta_ms=110.63109
i=5 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4341410024968, t_end=4341521097284, delta_ms=111.072316
i=6 inds='all'
n_nodes: 145 serialized: 144
Wait timings
Compute timings: t_begin=4341526443206, t_end=4341637783152, delta_ms=111.339946
i=7 inds='all'
n_nodes: 145 serializ

In [41]:
print(len_operations_data)
agent = BaselineAgent("neural",len_operations_data,max_gr_width, num_device,  device="cpu" if ncuda==0 else "cuda")
trainer = AgentTrainer(agent)
best_model = None
best_res = 1e14

24


In [42]:
# num_episodes = 300

# for_gen = GraphRunner(file, ncpus, ncuda, num_episodes +((num_episodes + 19)//20) * 5, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
# for_gen.run(starpu_home=STARPU_HOME)
# _ = for_gen.skip_initialization_routine()
# BEST_TIME = 0
# # BEST_TIME = get_etalon(file, gr)
# for i_episode in range(num_episodes):
#     if (i_episode % 20 == 0):
#         BEST_TIME = get_etalon(file, for_gen)
#     # Initialize the environment and get its state
#     my_slice = graph_slice(for_gen)
#     agent.schedule(my_slice)
#     cur_t = my_slice.get_timings()
    
#     for el in agent.frc:
#         reward = torch.tensor(my_slice.get_time(el[3]) / (BEST_TIME), dtype=torch.float32, device=device).unsqueeze(0)
#         agent.memory.push(el[0], el[1], el[2], reward)
#     trainer.step(agent = agent, timing = cur_t)
#     #if (COUNT_IT == INFO_SZ):
#       #  print("good_st = ", good_steps)
#       #  print("normal metric = ", get_timings_policy(5, for_gen) / get_timings_for_acc(5, for_gen))
#       #  COUNT_IT = 0
#       #  INFO_MEAN = 0
#  #       if done:
#   #          episode_durations.append(t + 1)
#    #         plot_durations()
#     #        break

In [43]:
file = f"{HOMEDIR}/graph_tracer/traced_graph_32_layers.json"
num_episodes = 150
for_gen = GraphRunner(file, ncpus, ncuda, num_episodes+((num_episodes + EPOCH_BEFORE_BEST_REC - 1)//EPOCH_BEFORE_BEST_REC) * 5, True, verbose=verbose, home_dir=HOMEDIR)
for_gen.run()
_ = for_gen.skip_initialization_routine()

BEST_TIME = 0
TRAIN_FOR_TRACE = 20

epi_to_ac = []
epi_to_score = []
for i_episode in range(num_episodes):
    if (i_episode % EPOCH_BEFORE_BEST_REC == 0):
        BEST_TIME = get_etalon(file, for_gen)
    # Initialize the environment and get its state
    my_slice = graph_slice(for_gen)
    ac = agent.schedule(my_slice)
    cur_t = my_slice.get_timings()
    epi_to_score.append(cur_t / BEST_TIME)
    epi_to_ac.append(ac)
    if (cur_t / BEST_TIME < best_res):
        best_res = cur_t /BEST_TIME
        best_model = agent
    for el in agent.frc:
        reward = torch.tensor(my_slice.get_time(el[3]) / (BEST_TIME), dtype=torch.float32, device=device).unsqueeze(0)
        agent.memory.push(el[0], el[1], el[2], reward)
    for  i in range(TRAIN_FOR_TRACE):
        trainer.step(agent = agent, timing = cur_t)
    #if (COUNT_IT == INFO_SZ):
      #  print("good_st = ", good_steps)
      #  print("normal metric = ", get_timings_policy(5, for_gen) / get_timings_for_acc(5, for_gen))
      #  COUNT_IT = 0
      #  INFO_MEAN = 0
 #       if done:
  #          episode_durations.append(t + 1)
   #         plot_durations()
    #        break

Run process:  python3 /workspace/graph_dataset.py --file /workspace/graph_tracer/traced_graph_32_layers.json --ncpus 4 --ncuda 0 --niters 190 --manual_sampling
Process pid: 568


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


i=0 inds='all'
n_nodes: 2095 serialized: 2094
Wait timings
Compute timings: t_begin=4362678443422, t_end=4374421198715, delta_ms=11742.755293
i=1 inds='all'
n_nodes: 2095 serialized: 2094
Wait timings
Compute timings: t_begin=4374477532540, t_end=4386190342322, delta_ms=11712.809782
i=2 inds='all'
n_nodes: 2095 serialized: 2094
Wait timings
Compute timings: t_begin=4386249895902, t_end=4397983888439, delta_ms=11733.992537
i=3 inds='all'
n_nodes: 2095 serialized: 2094
Wait timings
Compute timings: t_begin=4398039639003, t_end=4409713637726, delta_ms=11673.998723
i=4 inds='all'
n_nodes: 2095 serialized: 2094
Wait timings
Compute timings: t_begin=4409769440247, t_end=4421491437080, delta_ms=11721.996833
2095
graph_gened


IndexError: index 12289 is out of bounds for axis 0 with size 12289

In [ ]:
# file = f"{HOMEDIR}/graph_tracer/traced_graph_8_layers.json"
# num_episodes = 100
# for_gen = GraphRunner(file, ncpus, ncuda, num_episodes+((num_episodes + 19)//20) * 5, True, verbose=verbose, home_dir=HOMEDIR)
# for_gen.run()
# _ = for_gen.skip_initialization_routine()

# BEST_TIME = 0
# for i_episode in range(num_episodes):
#     if (i_episode % 20 == 0):
#         BEST_TIME = get_etalon(file, for_gen)
#     # Initialize the environment and get its state
#     my_slice = graph_slice(for_gen)
#     agent.schedule(my_slice)
#     cur_t = my_slice.get_timings()
#     if (cur_t / BEST_TIME < best_res):
#         best_res = cur_t /BEST_TIME
#         best_model = agent
#     for el in agent.frc:
#         reward = torch.tensor(my_slice.get_time(el[3]) / (BEST_TIME * BEST_TIME), dtype=torch.float32, device=device).unsqueeze(0)
#         agent.memory.push(el[0], el[1], el[2], reward)
#     trainer.step(agent = agent, timing = cur_t)
#     #if (COUNT_IT == INFO_SZ):
#       #  print("good_st = ", good_steps)
#       #  print("normal metric = ", get_timings_policy(5, for_gen) / get_timings_for_acc(5, for_gen))
#       #  COUNT_IT = 0
#       #  INFO_MEAN = 0
#  #       if done:
#   #          episode_durations.append(t + 1)
#    #         plot_durations()
#     #        break

In [ ]:
# file = f"{HOMEDIR}/graph_tracer/traced_graph_16_layers.json"
# num_episodes = 60
# for_gen = GraphRunner(file, ncpus, ncuda, num_episodes+((num_episodes + 19)//20) * 5, True, verbose=verbose, home_dir=HOMEDIR)
# for_gen.run()
# _ = for_gen.skip_initialization_routine()
# for i_episode in range(num_episodes):
#     if (i_episode % 20 == 0):
#         BEST_TIME = get_etalon(file, for_gen)
#     # Initialize the environment and get its state
#     my_slice = graph_slice(for_gen)
#     agent.schedule(my_slice)
#     cur_t = my_slice.get_timings()
#     if (cur_t / BEST_TIME < best_res):
#         best_res = cur_t /BEST_TIME
#         best_model = agent
#     for el in agent.frc:
#         reward = torch.tensor(my_slice.get_time(el[3]) / (BEST_TIME * BEST_TIME), dtype=torch.float32, device=device).unsqueeze(0)
#         agent.memory.push(el[0], el[1], el[2], reward)
#     trainer.step(agent = agent, timing = cur_t)
#     #if (COUNT_IT == INFO_SZ):
#       #  print("good_st = ", good_steps)
#       #  print("normal metric = ", get_timings_policy(5, for_gen) / get_timings_for_acc(5, for_gen))
#       #  COUNT_IT = 0
#       #  INFO_MEAN = 0
#  #       if done:
#   #          episode_durations.append(t + 1)
#    #         plot_durations()
#     #        break

# Look, difference is 1.5%

It is really small, be carefull   
Just because testing graph setup is really small. Graph bigger, and if cant store in gpu - you will start feeling

For now we are not explored yet:
1. Tensor/Pipeline parallelism
2. Heterogenous setup (different gpus)
3. ofloading
4. backward passes and gradient chekpointing

In [ ]:
print(best_res)

In [ ]:
agent2 = BaselineAgent(strategy="dynamic", device="cpu" if ncuda==0 else "cuda")
trainer = AgentTrainer(agent)
gr = GraphRunner(file, ncpus, ncuda, 2 * 20, manual_sampling, verbose=verbose, home_dir=HOMEDIR)
gr.run(interactive=True, starpu_home=STARPU_HOME)
_ = gr.skip_initialization_routine()
timings_dynamic = []

for i in range(gr.niters // 2):
    # inds = get_subset_inds(rng, total_operations)
    inds = "all" # just use this if you want whole graph
    print(f"{i=} {inds=}")
    m = graph_slice(gr)
    best_model.schedule(m)
    timings = m.get_timings()
    graph = gr.next(inds=inds)

    # print("Nodes: ", len(graph.nodes))

    agent2.schedule(graph)

    gr.serialize_priorities(graph)
    timings2 = gr.wait_get_timings()
    
    timings_dynamic.append(timings / timings2)
print(timings_dynamic)

In [ ]:
import matplotlib.pyplot as plt
x = [i for i in range(len(epi_to_ac))]
plt.plot(x, epi_to_ac)
plt.show()
plt.plot(x, epi_to_score)
plt.show()

## Caveats

In [ ]:
# if metric collection done correctly - on every run you will have filled length_mcs
# it is NOT UPDATING in real time
# it is just statistics collected on collect_metrics=True and not updated since
# they are stored in STARPU_HOME/.starpu/sampling/codelets/*some_number*/
graph.nodes[list(graph.nodes.keys())[50]]